# AI502/KSE527, Homework 01

This file is made by Jaehoon Oh, which is modified based on https://github.com/floydhub/regression

In [1]:
import torch
import torch.nn as nn
import torch.utils.data

In [2]:
POLY_DEGREE = 4
torch.manual_seed(2020)
W_target = torch.randn(POLY_DEGREE, 1) * 5
b_target = torch.randn(1) * 5

In [3]:
def poly_desc(W, b):
    """Creates a string description of a polynomial."""
    result = ''
    for i, w in enumerate(W):
        result += '{:+.2f} x^{} '.format(w, len(W) - i)
    result += '{:+.2f}'.format(b[0])
    return result

In [4]:
print('==> The real function you should approximate:\t' + poly_desc(W_target.view(-1), b_target))

==> The real function you should approximate:	+6.19 x^4 -4.80 x^3 +7.71 x^2 -2.04 x^1 +4.40


---

In [5]:
def make_features(x):
    """Builds features i.e. a matrix with columns [x^4, x^3, x^2, x^1]."""
    x = x.unsqueeze(1)
    return torch.cat([x ** (POLY_DEGREE+1-i) for i in range(1, POLY_DEGREE+1)], 1)

In [6]:
def f(x):
    """Approximated function."""
    return x.mm(W_target) + b_target[0]

In [7]:
def get_dataset(dataset_size):
    """Builds a batch i.e. (x, f(x)) pair."""
    random = torch.randn(dataset_size)
    x = make_features(random)
    y = f(x)
    dataset = list(zip(x, y))
    return dataset


In [ ]:
dataset = get_dataset(200) # you can make as many as dataset as you want
print(dataset)

---

In [9]:
num_epochs = 500
batch_size = 50
learning_rate = 0.1
criterion = nn.SmoothL1Loss()

In [10]:
dataset_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

---

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(W_target.size(0), 1)
        
        # For fixing the initial weights and bias
        self.fc.weight.data.fill_(0.)
        self.fc.bias.data.fill_(0.)
        
    def forward(self, x):
        output = self.fc(x)
        return output

In [12]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("running on GPU")
else:
    device = torch.device("cpu")
    print("running on CPU")

running on GPU


---

In [21]:
import time
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import interp1d

MODEL_NAME = f"model-{int(time.time())}"
print(MODEL_NAME)

LOSS = []
EPOCHS = []

def fit(model,loader,criterion,learning_rate,num_epochs):
    model.train()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    with open("model_0.1.log", "a") as f:
        
        for epoch in range(num_epochs):
            for i, data in enumerate(loader):
                if torch.cuda.is_available():
                    x = data[0].type(torch.FloatTensor).cuda()
                    y = data[1].type(torch.FloatTensor).cuda()
                else:
                    x = data[0].type(torch.FloatTensor)
                    y = data[1].type(torch.FloatTensor)   

                y_hat = model(x)
                loss = criterion(y_hat, y)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                #print(y)
                #print(y_hat)

                LOSS.append(loss)
                EPOCHS.append(epoch)
                
            fig = plt.figure() #for multiple figurs, for one thing just use plt.plot(times, accuracies) and plt.show()
            plt.figure(figsize=(20,20))
            plt.rcParams.update({'font.size': 22})

            f = interp1d(EPOCHS, LOSS)

            #define axis, each axis is a graph
            ax1 = plt.subplot2grid((2,1), (0,0))
            ax2 = plt.subplot2grid((2,1), (1,0), sharex=ax1)

            ax1.plot(EPOCHS, LOSS, label="loss")
            ax1.legend(loc=2)
            ax2.plot(EPOCHS, f(EPOCHS), label="loss_smoothed")
            ax2.legend(loc=2)   
            plt.show()
                
            if epoch % 100 == 0:
                y = y.cpu().detach().numpy()
                y_hat = y_hat.cpu().detach().numpy()
                fig = plt.figure() #for multiple figurs, for one thing just use plt.plot(times, accuracies) and plt.show()
                plt.figure(figsize=(20,20))
                plt.rcParams.update({'font.size': 22})


                #define axis, each axis is a graph
                ax1 = plt.subplot2grid((2,1), (0,0))
                #ax2 = plt.subplot2grid((2,1), (1,0), sharex=ax1)

                ax1.plot(y, label="actual")
                ax1.plot(y_hat, label="learned")
                ax1.legend(loc=2)

                #ax2.plot(x, LF, label="learned")
                #ax2.legend(loc=2)

                plt.show()

                #f.write(f"{MODEL_NAME}, {epoch}, {round(time.time(),3)},{round(float(loss), 4)}, {x}, {y}, {y_hat}\n")

model-1585900118


---

In [22]:
net = Net().cuda() if torch.cuda.is_available() else Net()
print('==> Initial function:\t' + poly_desc(net.fc.weight.data.view(-1), net.fc.bias.data))
print('==> Actual function:\t' + poly_desc(W_target.view(-1), b_target))

==> Initial function:	+0.00 x^4 +0.00 x^3 +0.00 x^2 +0.00 x^1 +0.00
==> Actual function:	+6.19 x^4 -4.80 x^3 +7.71 x^2 -2.04 x^1 +4.40


In [ ]:
# train
fit(net,dataset_loader,criterion,learning_rate,num_epochs)

In [24]:
print('==> Learned function:\t' + poly_desc(net.fc.weight.data.view(-1), net.fc.bias.data))
print('==> Actual function:\t' + poly_desc(W_target.view(-1), b_target))

==> Learned function:	+7.01 x^4 -4.66 x^3 +4.39 x^2 -1.57 x^1 +5.33
==> Actual function:	+6.19 x^4 -4.80 x^3 +7.71 x^2 -2.04 x^1 +4.40
